In [49]:
import numpy as np
from numpy import linalg as LA
from numpy import matlib
from numpy.linalg import matrix_power
from timeit import default_timer as timer

In [50]:
D = 1000
U = np.zeros((D, 9))
V = np.zeros((D, 9))

In [51]:
for i in range(0, 9):
    U[100*i:100*(i+2),i] = (np.random.rand(200,1)/10).reshape(200,);
    V[100*i:100*(i+2),i] = (np.random.rand(200,1)/10).reshape(200,);

In [52]:
A = np.matmul(U,V.transpose())
eigenvalues, eigenvectors = LA.eig(A)
max_eigenvalue = max(np.absolute(eigenvalues))
A = np.dot(0.8, np.divide(A, max_eigenvalue))

In [53]:
mu = np.random.rand(D,1)/D
w = 1

In [54]:
import math
def comp_lambda(cur_t, cur_event, last_t, lambdat,w,mu,A):
    lambda_comp = mu + (lambdat - mu)*(math.exp(-w * (cur_t - last_t)))
    if (cur_event):
        lambda_comp = lambda_comp + np.expand_dims(A[cur_event, :].T,axis=1)
    return lambda_comp

In [55]:
import random 

def generate_samples(w, mu, A, num_sequences, max_events_per_sequence):
    start_time = timer()
    hp_samples = []

    for i in range(1, num_sequences+1):
        t = 0
        timestamp_and_event = []
        lambdat = mu
        lambdat_sum = np.sum(lambdat)
        
        while len(timestamp_and_event) < max_events_per_sequence:
            rand_u = random.uniform(0, 1)
            dt = np.random.exponential(1/lambdat_sum)            
            lambda_ts = comp_lambda(t+dt, [], t, lambdat,w,mu,A);
            lambdats_sum = np.sum(lambda_ts);
                        
            if (rand_u > (lambdats_sum/lambdat_sum)):
                t = t+dt
                lambdat = lambda_ts

            else:
                u = random.uniform(0, 1) * lambdats_sum
                lambda_sum = 0
                
                d = 0
                for d in range(1, len(mu)):
                    lambda_sum = lambda_sum + lambda_ts[d]
                    if(lambda_sum >= u):
                        break
            
                lambdat = comp_lambda(t+dt, d, t, lambdat, w, mu, A)
                t = t+dt
                timestamp_and_event.append([t,d])

            lambdat_sum = np.sum(lambdat)
        
        hp_samples.append(timestamp_and_event[0:])
        
        if (i%10 == 0):
            print("samples = " + str(i)+ "/" + str(num_sequences) + ", time = " 
                  + "{:.2f}".format(timer() - start_time) + " sec.")
    
    return hp_samples

In [56]:
num_sequences = 250
max_events_per_sequence = 100
hawkes_process_samples = generate_samples(w, mu, A, num_sequences, max_events_per_sequence)

samples = 10/250, time = 1.09 sec.
samples = 20/250, time = 2.22 sec.
samples = 30/250, time = 3.38 sec.
samples = 40/250, time = 4.58 sec.
samples = 50/250, time = 5.73 sec.
samples = 60/250, time = 6.92 sec.
samples = 70/250, time = 8.09 sec.
samples = 80/250, time = 9.20 sec.
samples = 90/250, time = 10.37 sec.
samples = 100/250, time = 11.52 sec.
samples = 110/250, time = 12.77 sec.
samples = 120/250, time = 13.88 sec.
samples = 130/250, time = 15.01 sec.
samples = 140/250, time = 16.40 sec.
samples = 150/250, time = 17.80 sec.
samples = 160/250, time = 19.07 sec.
samples = 170/250, time = 20.43 sec.
samples = 180/250, time = 21.95 sec.
samples = 190/250, time = 23.42 sec.
samples = 200/250, time = 25.25 sec.
samples = 210/250, time = 26.85 sec.
samples = 220/250, time = 28.57 sec.
samples = 230/250, time = 30.27 sec.
samples = 240/250, time = 31.60 sec.
samples = 250/250, time = 32.96 sec.


In [57]:
'''
timestamp = 1x100 double
event = 1x100 event            
'''
print (len(hawkes_process_samples))
print (len(hawkes_process_samples[0]))
print (hawkes_process_samples[0])

250
100
[[2.229037467659969, 233], [3.436823202405772, 789], [3.8500135099582518, 745], [4.270400205575894, 816], [4.546569066007944, 821], [5.529007830133845, 885], [6.309214903433597, 862], [6.685206952097567, 831], [7.250035230814646, 99], [7.490669240483677, 813], [9.628439011861392, 307], [13.360906323367898, 334], [13.838986651412776, 488], [16.111771807414108, 265], [16.599772634259768, 291], [17.517549868295788, 877], [17.578846022163006, 126], [19.82993708080597, 150], [20.194813069359995, 420], [20.845646407162544, 419], [21.184000104744737, 457], [22.419856871890367, 503], [24.325220626870717, 906], [24.726981430133304, 213], [26.210902790351863, 300], [27.843339650273236, 495], [28.947362082641995, 778], [29.794697783582933, 17], [30.17467229064059, 772], [31.680920747604763, 870], [32.224418984500495, 7], [32.71754990985976, 31], [33.697995504215115, 174], [33.706477050593115, 129], [34.492065144728045, 209], [35.86060059641788, 339], [40.97168958936185, 165], [41.69076824

In [116]:
def kernel_g(dt, w):
    g = np.multiply(w, np.exp(np.multiply(-w, dt)))
    g[g > 1] = 0
    return g

In [117]:
def kernel_int_g(dt, w):
    G = np.subtract(1, np.exp(np.multiply(-w, dt)))
    G[G<0] = 0
    return G

In [98]:
def real_err(A, A_m):
    err_1 = np.divide(abs(A - A_m), A)
    err_2 = abs(A - A_m) 
    
    err_1 = np.nan_to_num(err_1)
    err_1 = convert_inf(err_1, np.isinf(err_1))
    
    err = np.multiply(err_1, double((A!=0)*1)) + np.multiply(err_2, double((A==0)*1))
    err = np.sum(err.flatten())/double(A.size)
    
    return err

In [99]:
def convert_inf(A, inf_A):
    for i in range(0, len(inf_A)):
        for j in range(0, len(inf_A[0])):
            if(inf_A[i][j]):
                A[i][j] = 0
    return A

In [100]:
def optimize_mu(hp_samples, D, w, rho, num_iter_1, num_iter_2, thold, real_A):
    A_m = np.random.rand(D,D);
    eigenvalues_m, eigenvectors_m = LA.eig(A_m)
    max_eigenvalue_m = max(np.absolute(eigenvalues_m))
    A_m = np.dot(0.8, np.divide(A_m, max_eigenvalue_m))
    
    # reshape may required in mu_m
    mu_m = np.divide(np.random.rand(D, 1), D)
    UL = np.zeros((D, D))
    ZL = np.zeros((D, D))
    US = np.zeros((D, D))
    ZS = np.zeros((D, D))
    
    for i in range(0, num_iter_1 + 1):
        rho = rho * 1.1
        
        for j in range(0, num_iter_2 + 1):
            print ("No. " + str(i + 1) + " outter while iteration | No. " 
                   + str(j + 1) +  " inner while iteration")
            A_m, mu_m, RelErr = update_mu(A_m, mu_m, hp_samples, UL, ZL, US, ZS, w, rho, D, real_A)
            Iteration_Err[j + 1, i + 1] = RelErr
        
        s, v, d = np.linalg.svd(np.add(A_m, US))
        v = np.subtract(v, thold / rho)
        v[v < 0] = 0
        ZL = s * v * d.T
        UL = UL + np.subtract(A_m, ZL) #changed A_m-ZL to np.subtract
        
        #idk looks good to me lol
        tmp = np.subtract(abs(np.add(A_m, US)), thold / rho) # may have error 
        
        tmp[tmp < 0] = 0
        ZS = (np.multiply(np.sign(np.add(A_m, US)), tmp))
        US = np.add(US, np.subtract(A_M, ZS))
        
    return A_m, mu_m, Iteration_Err

In [128]:
def update_mu (A_m, mu_m, hp_samples, UL, ZL, US, ZS, w, rho, D, real_A):
    num_samples = len(hp_samples)
    mu_numerator = np.zeros((D, 1))
    
    C = np.zeros((len(A_m), len(A_m[0])))
    A_Step = np.add(np.zeros((len(A_m), len(A_m[0]))), 2 * rho)
    B = np.add(np.add(np.zeros((len(A_m), len(A_m[0]))), np.multiply(rho, np.subtract(UL, ZL))), 
               np.multiply(rho, np.subtract(US, ZS)))
    
    for s in range(0, num_samples):
        cur_hp_samples = hp_samples[s]
        timestamp = [i[0] for i in cur_hp_samples]
        event = [i[1] for i in cur_hp_samples]
        tc = timestamp[len(timestamp) - 1]
        nc = len(event)
        dt = np.subtract(tc, timestamp)
        
        for i in range(0, nc):
            ui = event[i]
            ti = timestamp[i]
            int_g = kernel_int_g(dt, w)
            
            # Todo: modify matrix B (Incomplete)
            # B(ui,:) = B(ui,:) + double(A_m(ui,:)>0).*repmat(int_g(i),[1,D]);            
            B[ui,:] = B[ui,:] + np.multiply((A_m[ui,:]>0).astype(float), np.matlib.repmat(int_g[i], 1, D))
            
            pii = []
            pij = []
            ag_arr = []
            
            if (i > 0):
                tj = timestamp[0 : i]
                uj = event[0 : i]
                kn_g = kernel_g(np.subtract(ti, tj), w)
                ag_arr = np.multiply(A_m[uj, ui], kn_g.T)
                
            pii = np.divide(mu_m[ui], mu_m[ui] + sum(ag_arr))
            
            if(i > 0):
                pij = np.divide(ag_arr, mu_m[ui] + sum(ag_arr))
                if (len(pij) != 0 and sum(pij) > 0):
                    for j in range(0, len(uj)):
                        uuj = uj[j]
                        C[uuj, ui] = C[uuj, ui] - pij[j, :] ## (Incomplete) Question: what we have at the end ??? value or vector ??
            
            mu_numerator[ui] = np.add(mu_numerator[ui], pii)
            
    mu = np.divide(mu_numerator, np.add(np.zeros((D, 1)), tc))
    sqrt_eq = math.sqrt(np.subtract(matrix_power(B, 2), np.multiply(4, np.multiply(A_step, C))))
    A  = np.divide(np.add(np.multiply(-1, B), sqrt_eq), np.multiply(2, A_Step))
    RelErr = real_err(real_A, A)
    
    print ("non-zero in mu = " + np.count_nonzero(mu))
    print ("non-zero in C = "  + np.count_nonzero(C))
    print ("non-zero in B = "  + np.count_nonzero(B) + ", non-zero in sqrt = " + np.count_nonzero(sqrt_eq))
    print ("real error = " + "{:.4f}".format(RelErr) + ", correlation = " + "{:.4f}".format()  # (Incomplete)
           + "#non-zero in A = " + np.count_nonzero(A))
            
    A = np.nan_to_num(A)
    A = convert_inf(A, np.isinf(A))
    
    return A, mu, RelErr

In [129]:
#start optimization
num_iter_1 = 2;    # Number of Iteration of the First while loop (while k = 1,2 ...)
num_iter_2 = 7;    # Number of Iteration of the Second while loop (while not converge)

rho = 0.09;
thold = 1;         # thershold value

In [130]:
[x,y,Iteration_Err] = optimize_mu(hawkes_process_samples,D,w,rho,num_iter_1,num_iter_2,thold,A);

No. 1 outter while iteration | No. 1 inner while iteration


IndexError: too many indices for array